In [ ]:
import re
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def process_one(filename):
    p1=re.compile("Longest Blockchain Length: (\d*)")
    p2=re.compile("generate a block with parent height: (\d*)")
    last_longest = None
    attack_forking_start = None
    attack_success = None
    with open(filename) as fin:
        for i, line in enumerate(fin):
            result = p1.search(line)
            if result is not None:
                last_longest = (i, int(result.group(1)) )
            result = p2.search(line)
            if result is not None:
                parent = int(result.group(1))
                if attack_forking_start is None:
                    attack_forking_start = parent
                attack_forking_length = parent+1
                if attack_forking_length >= last_longest[1]:
                    #attack success
                    attack_success = attack_forking_length
    return attack_forking_start, attack_success

In [ ]:
def process_mining_rate(filename):
    p1=re.compile("Total Number of PoS Blocks in Blockchain: (\d*)")
    p2=re.compile("Total Number of PoW Blocks in Blockchain: (\d*)")
    mining_rate_pos = None
    mining_rate_pow = None
    with open(filename) as fin:
        for i, line in enumerate(fin):
            result = p1.search(line)
            if result is not None:
                r = int(result.group(1))
                mining_rate_pos = r
            result = p2.search(line)
            if result is not None:
                r = int(result.group(1))
                mining_rate_pow = r
    return mining_rate_pos, mining_rate_pow

In [ ]:
def generate_rates(filepath):
    ssum=0.0
    wsum=0.0
    cnt=0
    matrix = np.zeros((11,11))
    for betas in range(11):
        for betaw in range(11):
            if betas+betaw==0:
                continue
            b1=betas/10
            b2=betaw/10
            s, w = process_mining_rate(filepath+str(betas)+'-'+str(betaw)+'-attacker.log')
            ssum+=s
            wsum+=w
            cnt+=1
    return ssum/cnt,wsum/cnt

In [ ]:
def generate_matrix(filepath):
    matrix = np.zeros((11,11))
    for betas in range(11):
        for betaw in range(11):
            if betas+betaw==0:
                continue
            b1=betas/10
            b2=betaw/10
            attack_forking_start, attack_success = process_one(filepath+str(betas)+'-'+str(betaw)+'-attacker.log')
            if attack_success is not None:
                matrix[betas][betaw] = attack_success-attack_forking_start
    return matrix

In [ ]:
def generate_array(filepath):
    # for pure pos / pow
    matrix = np.zeros((11,))
    for betas in range(11):
        if betas==0:
            continue
        b1=betas/10
        attack_forking_start, attack_success = process_one(filepath+str(betas)+'-attacker.log')
        if attack_success is not None:
            matrix[betas] = attack_success-attack_forking_start
    return matrix

In [ ]:
pure_pos = np.zeros((11,))
for i in range(10):
    arr1=generate_array('./private_attack_experiment_ouro'+str(i)+'/')
    pure_pos+=arr1
pure_pos/=10

In [ ]:
minotaur = np.zeros((11,11))
for i in range(0,10):
    matrix=generate_matrix('./private_attack_experiment_mino'+str(i)+'/')
    minotaur+=matrix
minotaur/=10

In [ ]:
minotaur_max = np.ceil(np.max(minotaur))
norm = matplotlib.colors.Normalize(vmin=0, vmax=minotaur_max)

grid = dict(height_ratios=[minotaur.shape[0],1], width_ratios=[1,minotaur.shape[0], 0.5 ])
fig, axes = plt.subplots(ncols=3, nrows=2, gridspec_kw = grid, figsize=[6.4*1.2, 5.9*1.2])
axes[0,1].imshow(minotaur, cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[0,0].imshow(pure_pos.reshape((11,1)), cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[1,1].imshow(pure_pos.reshape((1,11)), cmap='Reds', aspect='auto', interpolation='nearest', origin='lower', norm=norm)
axes[1,0].axis("off")
axes[1,2].axis("off")
for ax in [axes[0,1], axes[0,0], axes[1,1]]:
    ax.set_xticks([]); ax.set_yticks([])
fig.colorbar(matplotlib.cm.ScalarMappable(cmap="Reds", norm=norm), cax=axes[0,2])
# plt.imshow(minotaur, cmap='Reds', interpolation='nearest', origin='lower')
axes[1,1].set_xlabel(r"$\beta_w$")
axes[0,0].set_ylabel(r"$\beta_s$")
axes[0,1].set_xticks(np.linspace(0,10,6), np.round(np.linspace(0,1,6),2))
axes[0,1].set_yticks(np.linspace(0,10,6), np.round(np.linspace(0,1,6),2))
axes[0,1].set_title("Minotaur")
axes[0,0].set_title("Ouroboros\n Praos")
axes[1,2].set_title("Bitcoin", x=0.5, y=0.1)
# fig.set_aspect(1)
# plt.colorbar()
plt.savefig("tmp10.pdf", bbox_inches='tight')

In [ ]:
lists=[]
listw=[]
for i in range(0,10):
    s,w=generate_rates('./private_attack_experiment_mino'+str(i)+'/')
    lists.append(s)
    listw.append(w)
print('block rates', np.mean(lists)/240, np.mean(listw)/240)